This tutorial is based on the [legacy FEniCS tutorial](https://fenicsproject.org/pub/tutorial/html/._ftut1005.html) and the [FEniCSx tutorial on the deflection of a membrane](https://jsdokken.com/dolfinx-tutorial/chapter1/membrane.html).

## Gmsh installation

The built-in FEniCSx meshes are well suited for testing and simple academic problems, but many geometries in real life have more complicated shapes. Gmsh is a free and open-source mesh generator that can handle more complex geometries.

To add Gmsh to your Conda environment `fenicsx-env`, first make sure you have this environment activated and then

```bash
conda install gmsh python-gmsh
```


## Temperature distribution in a room

Our first FEniCSx program for the Poisson equation targeted a
simple test problem where we could easily verify the
implementation. We now turn our attention to a physically more
relevant problem with solutions of somewhat more exciting shape.

We want to compute the temperature $T(x,y)$ inside a (two-dimensional) room 
that is equipped with space heaters. The appropriate PDE model is

\begin{equation}
-k\Delta T = f\quad\hbox{in }\Omega
\tag{1}
\end{equation}

Here, $k = 2.4$ (in suitable units) is the effective heat conductivity of air,
and $f$ is the external heat source from the heaters.
Initially we assume that the outside walls of the room have a fixed
temperature of 10°C, implying $T=10$ as a boundary condition.

A localised heat source can be modeled as a piecewise function:

\begin{equation}
f(x,y) = \begin{cases}
2 & \text{if } 0\leq x \leq 0.3\\
0 & \text{otherwise}
\end{cases}.
\tag{2}
\end{equation}

### Creating geometries with Gmsh

To create the computational geometry, we use the Python API of Gmsh. We start by importing the gmsh module and initialising it.

In [25]:
import gmsh
gmsh.initialize()

The next step is to create the room layout and start the computations by the Gmsh Open Cascade CAD kernel (`occ`), to generate the relevant underlying data structures.

Some common two-dimensional shapes can be generated with the commands
* `addDisk`
* `addRectangle`

The first three arguments of `addDisk` are the $x$, $y$ and $z$ coordinate of the center of the disk, while the two last arguments are the $x$-radius and $y$-radius.

The first three arguments of `addRectangle` are 📝_______________________, while the two last arguments are 📝_______________________.

In [26]:
disk = gmsh.model.occ.addDisk(0, 0, 0, 10, 10)
rectangle1 = gmsh.model.occ.addRectangle(-10, 5, 0, 20, 5)
rectangle2 = gmsh.model.occ.addRectangle(-10, -10, 0, 20, 5)
help(gmsh.model.occ.addRectangle)
gdim = 2 # geometric dimension of this model

Help on function addRectangle in module gmsh:

addRectangle(x, y, z, dx, dy, tag=-1, roundedRadius=0.0)
    gmsh.model.occ.addRectangle(x, y, z, dx, dy, tag=-1, roundedRadius=0.)

    Add a rectangle in the OpenCASCADE CAD representation, with lower left
    corner at (`x', `y', `z') and upper right corner at (`x' + `dx', `y' +
    `dy', `z'). If `tag' is positive, set the tag explicitly; otherwise a new
    tag is selected automatically. Round the corners if `roundedRadius' is
    nonzero. Return the tag of the rectangle.

    Return an integer.

    Types:
    - `x': double
    - `y': double
    - `z': double
    - `dx': double
    - `dy': double
    - `tag': integer
    - `roundedRadius': double



📝 Make a sketch of these three shapes on paper!

To form more complex shapes out of the basic ones, Gmsh provides commands such as
* `cut` (set difference)
* `fuse` (union)
* `intersect` (intersection)

In [27]:
room = gmsh.model.occ.cut([(gdim, disk)], [(gdim, rectangle1), (gdim, rectangle2)])

📝 In your sketch on paper, shade the region that results from this operation.

Next, we need to transfer these data from the Open Cascade kernel to Gmsh.

After that, we make the room a physical surface, such that it is recognised by Gmsh when generating the mesh. To this end, we first extract all two-dimensional pieces ("entities") from the geometric model and then collect them in a physical group. As a surface is a two-dimensional entity, we add `gdim` (i.e. `2`) as the first argument, the list of entity tags the room is composed of as the second argument, and the desired physical tag as the last argument. At a later stage in this course, we will get into when this tag matters.

In [29]:
gmsh.model.occ.synchronize()
surface_entities = [entity[1] for entity in gmsh.model.getEntities(dim=gdim)]
gmsh.model.addPhysicalGroup(gdim, surface_entities, tag=1)

Error   : Physical surface 1 already exists


Exception: Physical surface 1 already exists

Finally, we generate the two-dimensional mesh. We set a uniform mesh size by modifying the Gmsh options.

In [ ]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)
gmsh.model.mesh.generate(gdim)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Ellipse)
Info    : [ 70%] Meshing curve 4 (Line)
Info    : [ 90%] Meshing curve 5 (Ellipse)
Info    : Done meshing 1D (Wall 0.000466255s, CPU 0.000614s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.601838s, CPU 0.588862s)
Info    : 22646 nodes 45295 elements


### Using Gmsh models in FEniCSx

We will import the Gmsh-mesh directly from Gmsh into DOLFINx via the `dolfinx.io.gmshio` interface.

The communicator and rank arguments will play a role later on when we consider computing in parallel.

We will get two mesh tags, one for cells marked with physical groups in the mesh and one for facets marked with physical groups. As we did not add any physical groups of dimension `gdim-1`, there will be no entities in the `facet_markers`.

❓ For what kind of boundary conditions would it be interesting to have physical groups of dimension `gdim-1` in the Gmsh model?

❗ 📝 ______________________________________

In [ ]:
from mpi4py import MPI
import numpy as np
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, conditional, And, gt, lt, TrialFunction, TestFunction, inner, grad, dx, ds
from pathlib import Path

domain, cell_markers, facet_markers = io.gmshio.model_to_mesh(
    model=gmsh.model,
    comm=MPI.COMM_WORLD,
    rank=0,
    gdim=gdim
)

We define the function space and constant terms as in the previous tutorial.

In [ ]:
V = fem.functionspace(domain, ("P", 1))

k = fem.Constant(domain, 2.4)
alpha = fem.Constant(domain, 1.)
T0 = fem.Constant(domain, 10.)

The piecewise heat source function is represented using the unified form language UFL:

In [ ]:
x = SpatialCoordinate(domain)
f = conditional(And(gt(x[0], 0), lt(x[0],0.25)), fem.Constant(domain, 2.), fem.Constant(domain, 0.))

The boundary conditions are set using the topological information.

In [ ]:
tdim = domain.topology.dim # topological dimension of the mesh
fdim = tdim - 1 # facet dimension
domain.topology.create_connectivity(fdim, tdim) # what facets are connected to which cells
boundary_facets = mesh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(
    V=V,
    entity_dim=1,
    entities=boundary_facets
)

#bc = fem.dirichletbc(T0, boundary_dofs, V)

Now the problem formulation can be specified in the well-known fashion:

In [ ]:
T = TrialFunction(V)
v = TestFunction(V)
#a = k * inner(grad(T), grad(v)) * dx # with dirichlet
a = k * inner(grad(T), grad(v)) * dx - (alpha * T * v) * ds # with neumann
L =(f * v ) * dx - (alpha * T0 * v) * ds
problem = LinearProblem(a, L, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
Th = problem.solve()
Th.name = "Temperature"

Finally, we export the data in VTX format:
* the heat source $f$ as a piecewise constant function
* the numerical solution for $T$ as a piecewise linear function

In [ ]:
Q = fem.functionspace(domain, ("DP", 0))
expr = fem.Expression(f, Q.element.interpolation_points())
heat_source = fem.Function(Q)
heat_source.interpolate(expr)
heat_source.name = "Heat Source"

results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
with io.VTXWriter(MPI.COMM_WORLD, results_folder / "heat_source.bp", [heat_source], engine="BP4") as vtx:
    vtx.write(0.0)
with io.VTXWriter(MPI.COMM_WORLD, results_folder / "temperature.bp", [Th], engine="BP4") as vtx:
    vtx.write(0.0)